# Accessibility maps

This notebook creates maps to cartographically represent the accessibility estimations. These maps are generated using five intervals with a natural break approach to guarantee the data is correctly visualized. Additionally, a single interval dedicated to cero values is created to enhance interpretation.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from geo_northarrow import add_north_arrow
from shapely.geometry.point import Point
from mapclassify import NaturalBreaks
import numpy as np

In [ ]:
# Eliminate warning of KMeans
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# Import the names of the territories
l_names = pd.read_csv('../data/input/table/mpios_names.txt',header=None)
l_names = list(l_names[0])

# Types of codes
codes_names_list = ['health','sport','education','financial','cultural','parks']

# Import the population data
DANE_data = pd.read_csv('../data/input/table/DANE_2018_personas_manz.txt',low_memory=False)
pop = DANE_data[['MANZ_CCNCT','poblacion']]

In [ ]:
for m in l_names:
    
    # Import the city's blocks
    blocks = gpd.read_file(f'../data/output/shape/blocks/{m}_blocks.shp')
    blocks = blocks[['MANZ_CCNCT','geometry']]
    # Combine the population and the blocks
    blocks = blocks.set_index('MANZ_CCNCT').join(pop.set_index('MANZ_CCNCT'))
    blocks = blocks.reset_index()
    blocks = blocks.rename(columns={'poblacion':'p_(h)'})
    # Change the NaN values for 0.0
    blocks = blocks.fillna(0.0)
    # Calculate the proportion of population
    blocks['pp'] = blocks['p_(h)']/blocks['p_(h)'].sum()
    # Territory density
    perimeter = gpd.read_file(f'../data/output/shape/perimeters/{m}_perimeter.shp')
    perimeter = perimeter.to_crs('epsg:32618')
    perimeter['A'] = perimeter.area/1e6
    # Calculate the total population
    tot_pop = blocks['p_(h)'].sum()
    # Create a list with the accessibility type names: health, sport, etc.
    A_i_names = []
    # Create a list with the normalized accessibility type names
    N_i_names = []
    # Create a loop to agregate each accessibility measure to the blocks
    for c in codes_names_list:
        # Import the accessibility DataFrame
        acc_i_df = pd.read_csv(f'../data/output/table/accessibility_dfs/contour/accessibility_i_contour_15min_{m}_{c}.txt')
        # Filter the accessibility DataFrame
        acc_i_df = acc_i_df[['MANZ_CCNCT','Acc_i']]
        A_i_names.append(f'A_i_{c[:3]}')
        N_i_names.append(f'N_i_{c[:3]}')
        # Rename the accessibility column with the respective type
        acc_i_df = acc_i_df.rename(columns={'Acc_i':f'A_i_{c[:3]}'})
        blocks = blocks.merge(acc_i_df,on='MANZ_CCNCT',how='left')
        blocks = blocks.fillna(0.0)
    
    # Eliminate blocks with 0 population
    blocks_norm = blocks.copy()
    blocks_norm = blocks_norm.drop(blocks_norm[blocks_norm['p_(h)']==0].index).reset_index(drop=True)
    
    for c in codes_names_list:
        
        blocks_acc = blocks_norm.copy()
        
        if blocks_acc[f'A_i_{c[:3]}'].sum() == 0:
            fig, ax = plt.subplots(figsize=(15, 10))
            blocks.plot(ax=ax, color='lightgray')
            
            blocks_acc.plot(ax=ax, color='#fde725', legend=True)
            from matplotlib.patches import Rectangle
            legend_handles = [
                Rectangle((0, 0), 1, 1, facecolor='#fde725', edgecolor='none', label='0.00')
            ]
            plt.legend(handles=legend_handles, loc='lower right', title='Accessibility')
            # Scale for epsg:4326
            points = gpd.GeoSeries([Point(-73.5, 40.5), Point(-74.5, 40.5)], crs=4326)  # Geographic WGS 84 - degrees
            points = points.to_crs(32618)
            distance_meters = points[0].distance(points[1])
            scale = ScaleBar(dx=distance_meters,location='lower left')
            ax.add_artist(scale)
            # North arrow
            add_north_arrow(ax, scale=.4, xlim_pos=.1, ylim_pos=.9, color='#000', text_scaler=3, text_yT=-1.25)
            plt.title(f'{m} map - {c} accessibility')
            plt.axis('off')
            plt.tight_layout()
            plt.savefig(f'../data/output/image/accessibility/contour_/{m}_accessibility_{c}.png')
            plt.close()
            print(f'{m} does not have {c} accessibility')
            blocks_acc = blocks_acc[['MANZ_CCNCT','p_(h)','pp',f'A_i_{c[:3]}','geometry']]
            blocks_acc.to_file(f'../data/output/shape/accessibility/contour_/accessibility_i_contour_15min_{m}_{c}.shp')
            continue

        fig, ax = plt.subplots(figsize=(15, 10))
        blocks.plot(ax=ax, color='lightgray')

        # Crear una columna que marque los valores igual a 0
        blocks_acc[f'A_i_cat_{c[:3]}'] = blocks_acc[f'A_i_{c[:3]}'].apply(lambda x: 0 if x == 0 else 'not_0')

        # Clasificar los valores no cero usando natural_breaks
        not_0_values = blocks_acc.loc[blocks_acc[f'A_i_{c[:3]}'] != 0, f'A_i_{c[:3]}']
        classifier = NaturalBreaks(not_0_values, k=4)
        blocks_acc.loc[blocks_acc[f'A_i_{c[:3]}'] != 0, f'A_i_cat_{c[:3]}'] = classifier.yb + 1  # +1 para evitar conflictos con la categoría '0'

        # Definir el esquema de colores incluyendo la nueva categoría
        colors = plt.cm.viridis_r(np.linspace(0, 1, 5)).tolist()

        # Si no hay ceros, crear una geometria diminuta para poder graficar
        if len(not_0_values) == len(blocks_acc):
            last_register_index = len(blocks_acc)-1
            buff_around_first_register = blocks_acc.to_crs('epsg:32618').iloc[[0]].centroid.buffer(0.01).to_crs('epsg:4326').loc[0]
            blocks_acc.loc[last_register_index+1,f'A_i_cat_{c[:3]}'] = 0
            blocks_acc.loc[last_register_index+1,'geometry'] = buff_around_first_register

        # Graficar
        blocks_acc.plot(column=f'A_i_cat_{c[:3]}', legend=True, cmap=plt.cm.colors.ListedColormap(colors), ax=ax, 
                        legend_kwds={'loc': 'lower right', 'title': 'Accesibilidad'})

        if classifier.k == 4:
            legend = ax.get_legend()
            original_legend_handles = legend.legend_handles
            original_legend_labels = ['0.0000, 0.0000'] + [f'0.0000, {classifier.bins[0]:.4f}'] + [f'{classifier.bins[i]:.4f}, {classifier.bins[i+1]:.4f}' for i in range(len(classifier.bins)-1)]
            # Ajustar la leyenda para incluir el nuevo intervalo
            ax.legend(original_legend_handles, original_legend_labels, loc='lower right', title='Accessibility')
        else:
            legend = ax.get_legend()
            original_legend_handles = legend.legend_handles
            original_legend_labels = ['0.00'] + [f'{classifier.bins[i]:.2f}' for i in range(len(classifier.bins))]
            # Ajustar la leyenda para incluir el nuevo intervalo
            ax.legend(original_legend_handles, original_legend_labels, loc='lower right', title='Accessibility')

        # Scale for epsg:4326
        points = gpd.GeoSeries([Point(-73.5, 40.5), Point(-74.5, 40.5)], crs=4326)  # Geographic WGS 84 - degrees
        points = points.to_crs(32618)
        distance_meters = points[0].distance(points[1])
        scale = ScaleBar(dx=distance_meters,location='lower left')
        ax.add_artist(scale)
        # North arrow
        add_north_arrow(ax, scale=.4, xlim_pos=.1, ylim_pos=.9, color='#000', text_scaler=3, text_yT=-1.25)
        plt.title(f'{m} map - {c} accessibility')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'../data/output/image/accessibility/contour_/{m}_accessibility_{c}.png')
        plt.close()

        blocks_acc = blocks_acc[['MANZ_CCNCT','p_(h)','pp',f'A_i_{c[:3]}','geometry']]
        blocks_acc.to_file(f'../data/output/shape/accessibility/contour_/accessibility_i_contour_15min_{m}_{c}.shp')